The aim of this Notebook is to create a code that calculate simulations for the pricicng and hedging of derivatives defined by a Cox-Ross-Rubinstien (CRR) model

### Parameters of the model

The parameters of the models are the following:

In [ ]:
import os
import numpy as np
import scipy as scp
import pylab
import math
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx 
r = 0.5   # The interest rate of the riskless bond
p = 0.5   # The probability for the price at each period to go upwarwd
q = 1.0-p # The probability for the price at each period to go downwarwd
u = 2.0   # The level of increase of the price
d = 0.5   # The level of decrease of the price
T = 5     # Number of periods in the model
S0 = 1.0  # Value of the risky asset at time 0

### The non-arbitrage condition

##### 1) Recall the non-arbitrage conditon and create a test command which prints an error message in case in is not satisfied

The non-arbitrage condition is: 
$$ d < 1+r <u $$

In [ ]:
if d >= 1.0+r or 1.0+r >= u:
    print("The market allows arbitrage")
else:
    print("The market is arbitrage free")
    print("We define the unique (in the case of the Cox-Ross-Rubinstein modem) equivalent martingale measure through  p∗ (p_star)")
    p_star = (1.0+r-d)/(u-d)
    print('p_star = ',p_star)

### Simulating sample paths of the risky asset

In [ ]:
M = 10    # The number of simulated sample paths
S = np.zeros((M,T+1))

We create and initialize the first valuer of the stock $S$

In [ ]:
start = np.zeros(M)
for i in range(0,M):
    start[i]=S0
S[:,0]=start

##### 2) Simulate $M$ paths of the stock $S$

In [ ]:
B = np.random.binomial(1,p,T+1) # We simulate all the Bernouilli random variables

In [ ]:
def transform(B): # Transform a vector of O and 1 into a vector of u and d's
    temp1 = np.size(B)
    up_and_down = np.zeros(temp1)
    for i in range(0,temp1):
        if B[i]==1:
            up_and_down[i]=u
        else:
            up_and_down[i]=d
    return up_and_down

In [ ]:
def simulateS(m):
    B = np.zeros(m)
    up_and_down = np.zeros(m)
    for i in range(1,T+1):
        B = np.random.binomial(1,p,m)
        up_and_down = transform(B)
        S[:,i]=S[:,i-1]*up_and_down

In [ ]:
simulateS(M)

In [ ]:
t1=np.linspace(0,T,num=T+1)
plt.plot(np.transpose(S))
plt.grid()
plt.show()

### Another representation using graphs (Binary trees)

In [ ]:
def binomial_grid(n,s0):
    G=nx.Graph() 
    for i in range(0,n):
        j=-i+1
        while (j<i+2):
            G.add_edge((i,j),(i+1,j+1),weight=0.0)
            G.add_edge((i,j),(i+1,j-1),weight=0.0)
            j=j+2
    
    j=1
    for i in range(0,n):
        r=np.random.binomial(1,p,1)
        if r >0:
            G.add_edge((i,j),(i+1,j+1),weight=1.0)
            j=j+1
        else:
            G.add_edge((i,j),(i+1,j-1),weight=1.0)
            j=j-1
    
    posG={}
    lab={}
    for node in G.nodes():
        posG[node]=(node[0],node[1])
        if node[0]==0:
            lab[node]=s0
        i=node[0]
        j=node[1]
        if j>1:
            lab[node]=s0*(u**(j-1))
        elif j==1:
            lab[node]=s0
        else:
            lab[node]=s0*(d**(1-j))
    elarge=[(a,b) for (a,b,c) in G.edges(data=True) if c['weight'] >0.5]
    esmall=[(a,b) for (a,b,c) in G.edges(data=True) if c['weight'] <=0.5]
    nx.draw_networkx_edges(G,posG,edgelist=elarge,edge_color='blue',width=2)
    nx.draw_networkx_edges(G,posG,edgelist=esmall,style='dashed')
    nx.draw_networkx_labels(G,posG,lab,font_size=15,font_family='sans-serif')
    plt.show()

In [ ]:
binomial_grid(5,100.0)

### Pricing

##### Pricing of European Financial claims on the Cox-Ross-Rubinstien model

The aim of this section is to write a function "Price" which computes the (arbitrage) price at any time of a contract whose payoff is given by a mapping $g_T$ using the Dynamic Programming Principle as in Theorem 6.2.1.

we start with a plotting function for a Binary tree

In [ ]:
def plot_tree(g):
    pos={}
    lab={}
    
    for n in g.nodes():
        pos[n]=(n[0],n[1])
        lab[n]=float(int(g.node[n]['value']*1000))/1000 # This is just to print only with 10^-2 precision
    
    elarge=g.edges(data=True)
    nx.draw_networkx_edges(g,pos,edgelist=elarge)
    nx.draw_networkx_labels(g,pos,lab,font_size=15,font_family='sans-serif')
    plt.show()

We construct the graph of prices

In [ ]:
def graph_stock():
    S=nx.Graph()
    for i in range(0,T):
        j=-i+1
        while (j<i+2):
            S.add_edge((i,j),(i+1,j+1))
            S.add_edge((i,j),(i+1,j-1))
            j=j+2
            
    for n in S.nodes():
        
        if n[0]==0:
            S.node[n]['value']=S0
        i=n[0]
        j=n[1]
        if j>1:
            S.node[n]['value'] = S0*(u**(j-1))
        elif j==1:
            S.node[n]['value'] = S0
        else:
            S.node[n]['value'] = S0*(d**(1-j))
    return S

In [ ]:
S=nx.Graph()
S =graph_stock()
plot_tree(S)

##### 3) Compute the price of a European Call Option

In [ ]:
def European_call(x,K):
        return np.maximum(x-K,0.0)

In [ ]:
def compute_European_Call_prices(K):

    g = nx.Graph()
    price = nx.Graph()
    
    S = nx.Graph()
    
    S = graph_stock()

    for i in range(0,T):
            j=-i+1
            while (j<i+2):
                g.add_edge((i,j),(i+1,j+1))
                g.add_edge((i,j),(i+1,j-1))
                price.add_edge((i,j),(i+1,j+1))
                price.add_edge((i,j),(i+1,j-1))
                j=j+2

    for n in S.nodes():
        if n[0]==5:
            g.node[n]['value'] = European_call(S.node[n]['value'],K)
            price.node[n]['value'] = European_call(S.node[n]['value'],K)
    
    i=T-1
    while i>=0:
        j=-i+1
        while j<i+2:
            g.node[(i,j)]['value'] = g.node[(i+1,j+1)]['value']*p_star + g.node[(i+1,j-1)]['value']*(1.0-p_star)
            price.node[(i,j)]['value'] = g.node[(i,j)]['value']*((1.0+r)**(i-T))
            j=j+2
        i=i-1
        
    return price


Computation the price of a European Call option

In [ ]:
price = nx.Graph()

K=0.0

K = input("Enter the value of the strike K : ")

price_Call = compute_European_Call_prices(float(K))

plot_tree(S)
plot_tree(price_Call)

print('Value of the price at time 0 of the European Call option with strike ',K,' = ',price_Call.node[(0,1)]['value'])

##### 4) Compute the price of a European Put Option

In [ ]:
def compute_European_Put_prices(K):

    g = nx.Graph()
    price = nx.Graph()
    Call = nx.Graph()
    
    Call = compute_European_Call_prices(K)
    
    S = nx.Graph()
    
    S = graph_stock()

    for i in range(0,T):
            j=-i+1
            while (j<i+2):
                price.add_edge((i,j),(i+1,j+1))
                price.add_edge((i,j),(i+1,j-1))
                j=j+2

    for n in S.nodes():
        if n[0]==5:
            price.node[n]['value'] = European_call(K,S.node[n]['value'])    
    i=T-1
    while i>=0:
        j=-i+1
        while j<i+2:
            price.node[(i,j)]['value'] = Call.node[(i,j)]['value'] + K*(1.0+r)**(i-T) - S.node[(i,j)]['value']
            j=j+2
        i=i-1
        
    return price

In [ ]:
price_Put = nx.Graph()

K=0.0

K = input("Enter the value of the strike K : ")

price_Call = compute_European_Call_prices(float(K))

price_Put = compute_European_Put_prices(float(K))

plot_tree(S)
plot_tree(price_Call)
plot_tree(price_Put)

##### 5) Complete the codes above to compute a hedging strategy and simumate the associated wealth processes

In [ ]:
# Fill here

##### 6) We now deal with the case of American Options

We admit that the price at a given time $t$ of an American Option with respect to the family of payoffs $(G_t:=g_t^{Amer}(S_t))_t$ is given by:
$$ \nu_t = \max\left(G_t, (1+r)^{-1} \mathbb{E}^{\mathbb{P}^\ast}[\nu_{t+1}\vert \mathcal{F}_t]\right), \quad \nu_T:=G_T. $$

Deduce that at any time $t$, $\nu_t = f_t^{Amer}(S_t)$ with
$$ f_t^{Amer}(x):= \max\left(g_t^{Amer}(x),(1+r)^{-1}\left(f_{t+1}^{Amer}(x u) p^\ast + f_{t+1}^{Amer}(x d) (1-p^\ast)\right)\right), \quad f_T^{Amer}(x):=g_T^{Amer}(x).$$

Write a code to compute the graph of an American Call Option and compare it with the price of the European Call Option

In [ ]:
def compute_American_Call_prices(K):

    price = nx.Graph()
    
    S = nx.Graph()
    
    S = graph_stock()

    for i in range(0,T):
            j=-i+1
            while (j<i+2):
                price.add_edge((i,j),(i+1,j+1))
                price.add_edge((i,j),(i+1,j-1))
                j=j+2

    for n in S.nodes():
        if n[0]==5:
            price.node[n]['value'] = European_call(S.node[n]['value'],K)
    
    i=T-1
    while i>=0:
        j=-i+1
        while j<i+2:
            temp = price.node[(i+1,j+1)]['value']*p_star + price.node[(i+1,j-1)]['value']*(1.0-p_star)
            price.node[(i,j)]['value'] = np.maximum(European_call(S.node[(i,j)]['value'],K),temp/(1.0+r))
            j=j+2
        i=i-1
        
    return price

In [ ]:
price_American_Call = nx.Graph()

price_European_Call = nx.Graph()

K=0.0

K = input("Enter the value of the strike K : ")

price_European_Call = compute_European_Call_prices(float(K))

price_American_Call = compute_American_Call_prices(float(K))

plot_tree(S)
plot_tree(price_European_Call)
plot_tree(price_American_Call)

Write a code to compute the graph of an American Put Option and compare it with the price of the European Call Option

In [ ]:
def compute_American_Put_prices(K):

    price = nx.Graph()
    
    S = nx.Graph()
    
    S = graph_stock()

    for i in range(0,T):
            j=-i+1
            while (j<i+2):
                price.add_edge((i,j),(i+1,j+1))
                price.add_edge((i,j),(i+1,j-1))
                j=j+2

    for n in S.nodes():
        if n[0]==5:
            price.node[n]['value'] = European_call(K,S.node[n]['value'])
    
    i=T-1
    while i>=0:
        j=-i+1
        while j<i+2:
            temp = price.node[(i+1,j+1)]['value']*p_star + price.node[(i+1,j-1)]['value']*(1.0-p_star)
            price.node[(i,j)]['value'] = np.maximum(European_call(K,S.node[(i,j)]['value']),temp/(1.0+r))
            j=j+2
        i=i-1
        
    return price

In [ ]:
price_American_Put = nx.Graph()

price_European_Put = nx.Graph()

K=0.0

K = input("Enter the value of the strike K : ")

price_European_Put = compute_European_Put_prices(float(K))

price_American_Put = compute_American_Put_prices(float(K))

plot_tree(S)
plot_tree(price_European_Put)
plot_tree(price_American_Put)

In [ ]:
r=0.0

K=1.0

opti = nx.Graph()
Ame_Put = nx.Graph()

Ame_Put = compute_American_Put_prices(K)
    
S = nx.Graph()
    
S = graph_stock()

for i in range(0,T):
        j=-i+1
        while (j<i+2):
            opti.add_edge((i,j),(i+1,j+1))
            opti.add_edge((i,j),(i+1,j-1))
            j=j+2

for n in S.nodes():
    if n[0]==5:
        opti.node[n]['value'] = 0.0
    
i=T-1
while i>=0:
    j=-i+1
    while j<i+2:
        opti.node[(i,j)]['value'] = Ame_Put.node[(i,j)]['value']-European_call(K,S.node[(i,j)]['value'])
        j=j+2
    i=i-1

plot_tree(opti)    